###  Create a response table using qgrid to pick the data and pivot table to format

In [228]:
import numpy as np
import pandas as pd
import os
import gcsfs
import xarray as xr

In [308]:
import pandas as pd

import qgrid
#dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')
#dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

dfcat = pd.read_csv('/home/naomi/cmip6-zarr/csv/pangeo-cmip6-20200423-noQC.csv')

len(dfcat)
#dfcat[dfcat.activity_id=='CMIP'].experiment_id.unique()

260307

In [326]:

#Trond DR1
table_id = 'Omon'
exps = ['esm-hist','historical','ssp245','ssp370','ssp585','1pctCO2']
variables = ['chl','dissoc','dissic','o2sat','ocfriver','icfriver']

# Ag
#table_id = 'Amon'
#exps = ['amip','historical']
#variables = ['tas','tasmin','tasmax']
#models = ['UKESM1-0-LL']

#Ekat
table_id = 'Omon'
exps = ['historical','ssp585']
variables = ['no3','no3os','nh4','nh4os','po4','po4os','dfe','dfeos','rsntds','si','sios','dissoc',
             'intpoc','intpp','intppdiat','intppdiaz','intppmisc','intpppico','chlmisc',
             'chlmiscos','chldiatos','chlpico','phycos','phyc','phymisc','phymiscos','phydiatos',
             'phydiat','phydiaz','phydiazos','phypico','pp','ppos','zooc','zoocos','fgco2','spco2','dissicnatos','epc100']

models = 'all'


exps = ['ssp126', 'ssp245', 'ssp370', 'ssp585', 'ssp119', 'ssp434','ssp460', 'ssp534-over']
exps += ['1pctCO2', 'abrupt-4xCO2', 'amip', 'historical', 'piControl',
       'esm-hist', 'esm-piControl', 'ssp126', 'ssp245', 'ssp370',
       'ssp585', 'esm-piControl-spinup', 'piControl-spinup']

#exps = ['ssp585','historical']
#exps =['1pctCO2','abrupt-4xCO2','piControl']
#variables = ['tas'] #,'pr','prc']
#table_id = 'Amon'

#table_id = 'Omon'
#exps = ['historical','ssp585']
#variables = ['thetao','uo','vo']

#table_id = 'Ofx'
#variables = 'sftof'

#table_id = 'Omon'
#exps = ['1pctCO2']
#variables = ['zos']

#table_id = 'Amon'
#models = ['GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-2-G']
#variables = ['hus','ta','ua','va']

#models = ['GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-2-G']
#table_id = '3hr'
#exps = ['historical']
#variables = ['hus','ta','va','ua']
#variables = ['huss','pr','tas','uas','vas']
#members = ['r1i1p1f1']

table_id = 'Amon'
exps = ['highresSST-present','highresSST-future']
variables = ['pr']

table_id = '6hrPlevPt'
exps = ['highresSST-present']
variables = ['ua','va']

table_id = 'Omon'
exps = ['historical','ssp585']
variables = ['sos','thetao','sosga']
models = ['NorESM2-LM','NorESM2-MM']

#table_id = 'Amon'
#exps = ['ssp126','ssp245','ssp370','ssp585']
#variables = ['ts','psl']
#members = ['r1i1p1f1','r2i1p1f1','r3i1p1f1']
#models = ['CESM2']

#['esm-ssp585', 'ssp245-GHG', 'ssp245-aer', 'ssp245-nat',
#        'esm-ssp585-ssp126Lu', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

table_id = 'Amon'
exps = ['1pctCO2','abrupt-4xCO2','amip','historical','piControl','ssp585']
variables = ['tas','clivi','pr','clwvi','rlut','rlutcs','rsut','rsutcs']
models = 'all'
members = 'all'

In [327]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

if models == 'all':
    dfs = search_df(dfcat,variable_id=variables,experiment_id=exps,table_id=table_id)
else:
    dfs = search_df(dfcat,variable_id=variables,experiment_id=exps,table_id=table_id,source_id=models)

In [328]:
widget=qgrid.show_grid(dfs)
widget 

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [329]:
# save results
df = widget.get_changed_df()     # use this to save filtered dataframe
#df = dfs
# df = widget.get_selected_df()  # use this instead to save selected rows
df.source_id.unique(), len(df)

(array(['AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0',
        'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'ACCESS-CM2',
        'ACCESS-ESM1-5', 'E3SM-1-0', 'EC-Earth3-Veg', 'INM-CM4-8',
        'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL',
        'HadGEM3-GC31-MM', 'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR',
        'MRI-ESM2-0', 'GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-2-G',
        'CESM2-FV2', 'CESM2-WACCM-FV2', 'CESM2-WACCM', 'CESM2', 'NorCPM1',
        'NorESM2-LM', 'KACE-1-0-G', 'GFDL-CM4', 'GFDL-ESM4', 'NESM3',
        'SAM0-UNICON', 'MCM-UA-1-0', 'NorESM2-MM', 'CNRM-CM6-1-HR',
        'EC-Earth3', 'TaiESM1', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0',
        'GFDL-AM4', 'CanESM5-CanOE', 'E3SM-1-1-ECA', 'E3SM-1-1',
        'MPI-ESM-1-2-HAM', 'INM-CM5-0', 'GISS-E2-1-G-CC', 'CIESM',
        'AWI-ESM-1-1-LR', 'IITM-ESM', 'EC-Earth3-LR', 'NorESM1-F'],
       dtype=object),
 5192)

In [330]:
print(sorted(df.source_id.unique()),len(df.source_id.unique()))
#df

['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'EC-Earth3', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-AM4', 'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'MCM-UA-1-0', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1', 'UKESM1-0-LL'] 55


In [331]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# use this to get number of models (one per model)
dm = df[['experiment_id','source_id','member_id','variable_id']].groupby([
         'experiment_id','variable_id']).nunique()[['source_id']]
# use this to get models*members
#         'experiment_id','member_id','variable_id']).nunique()[['source_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='source_id',
                                 index=['variable_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
print('activity_id, table_id = ',df.activity_id.unique(),df.table_id.unique())
print(table)

activity_id, table_id =  ['CMIP' 'CFMIP' 'ScenarioMIP'] ['Amon']
experiment_id  1pctCO2  abrupt-4xCO2  amip  historical  piControl  ssp585
variable_id                                                              
clivi                0             0     0          42          1      27
clwvi               33            31    32          41         39      27
pr                  36            35    38          47         50      32
rlut                35            34    35          45         51      31
rlutcs              34            33    33          43         48      29
rsut                35            32    33          43         52      29
rsutcs              35            33    33          42         49      29
tas                 36            33    38          45         48      32


activity_id, table_id =  ['CMIP' 'CFMIP' 'ScenarioMIP'] ['Amon']
experiment_id  1pctCO2  abrupt-4xCO2  amip  historical  piControl  ssp585
variable_id                                                              
clivi               38            37    35          46         44      31
clwvi               38            38    34          45         45      29
pr                  45            46    42          51         54      36
rlut                43            43    38          49         51      33
rlutcs              41            41    36          47         48      31
rsut                41            41    37          47         53      31
rsutcs              40            40    36          47         49      30
tas                 47            47    42          51         56      36


activity_id, table_id =  ['CMIP' 'CFMIP' 'ScenarioMIP'] ['Amon']
experiment_id                 1pctCO2  abrupt-4xCO2  amip  historical  piControl  ssp585
source_id        variable_id                                                            
ACCESS-CM2       clivi              0             0     0           2          0       1
                 pr                 1             1     3           2          1       1
                 rlut               1             1     1           2          1       1
                 rlutcs             1             1     1           2          1       1
                 rsut               1             1     1           2          1       1
                 rsutcs             1             1     1           2          1       1
                 tas                1             1     3           2          1       3
ACCESS-ESM1-5    clivi              0             0     0           3          0       3
                 pr                 1        

In [244]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dm = df[['experiment_id','source_id','variable_id','member_id']].groupby([
         'experiment_id','member_id','variable_id']).nunique()[['source_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='source_id',
                                 index=['member_id','variable_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
print('activity_id, table_id = ',df.activity_id.unique(),df.table_id.unique())
print(table)

activity_id, table_id =  ['CMIP' 'CFMIP' 'ScenarioMIP'] ['Amon']
experiment_id           1pctCO2  abrupt-4xCO2  amip  historical  piControl  ssp585
member_id  variable_id                                                            
r101i1p1f1 clivi              0             0     0           1          0       0
           clwvi              1             0     0           1          1       0
           pr                 1             0     0           1          1       0
           rlut               1             0     0           1          1       0
           rlutcs             1             0     0           1          1       0
...                         ...           ...   ...         ...        ...     ...
r9i1p3f1   rlut               0             0     0           1          0       0
           rlutcs             0             0     0           1          0       0
           rsut               0             0     0           1          0       0
           rsutcs     

activity_id, table_id =  ['ScenarioMIP'] ['Amon']
experiment_id          ssp126  ssp245  ssp370  ssp585
source_id variable_id                                
CESM2     psl               3       3       2       3
          ts                3       3       2       3


activity_id, table_id =  ['CMIP' 'ScenarioMIP'] ['Omon']
experiment_id           historical  ssp585
source_id  variable_id                    
NorESM2-LM sos                   3       1
           thetao                1       1
NorESM2-MM sos                   1       1
           thetao                1       1


In [95]:
tab2 = table[(table['historical']>0)&(table.ssp585>0)]
print(tab2)
print(len(tab2))

KeyError: 'historical'

In [96]:
x = tab2.index.values

tab2.index.values, len(tab2.index.values)
y=[]
for tup in x:
    y += [tup[0]]
y

['ACCESS-CM2',
 'ACCESS-ESM1-5',
 'AWI-CM-1-1-MR',
 'BCC-CSM2-MR',
 'CAMS-CSM1-0',
 'CESM2-WACCM',
 'CIESM',
 'CNRM-CM6-1',
 'CNRM-CM6-1-HR',
 'CNRM-ESM2-1',
 'CanESM5',
 'CanESM5-CanOE',
 'EC-Earth3',
 'EC-Earth3-Veg',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'FIO-ESM-2-0',
 'GFDL-CM4',
 'GFDL-ESM4',
 'GISS-E2-1-G',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'INM-CM5-0',
 'IPSL-CM6A-LR',
 'KACE-1-0-G',
 'MCM-UA-1-0',
 'MIROC-ES2L',
 'MIROC6',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM2-LM',
 'NorESM2-MM',
 'UKESM1-0-LL']

In [206]:
dm = df[['table_id','experiment_id','source_id','variable_id','member_id']].groupby([
         'table_id','experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id','source_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('current CMIP6 datasets, number of ensemble members available for table_id = ',table_id)
#display(table)

current CMIP6 datasets, number of ensemble members available for table_id =  Amon


In [207]:
dm = df[['table_id','experiment_id','source_id','variable_id','member_id']].groupby([
         'table_id','experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)
if len(variables)>len(exps):
    table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['variable_id'],
                                 columns=['experiment_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('before request: number of ensemble members available')
print(df.source_id.unique(), len(df.source_id.unique()))
print(df.zstore.nunique())
display(table)

before request: number of ensemble members available
['CESM2'] 1
22


variable_id,psl,ts
experiment_id,,
ssp126,3,3
ssp245,3,3
ssp370,2,2
ssp585,3,3


In [208]:
# Chiara wants:
array(['CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'E3SM-1-0',
       'EC-Earth3-Veg', 'EC-Earth3', 'IPSL-CM6A-LR', 'MIROC-ES2L',
       'MIROC6', 'MRI-ESM2-0', 'CESM2-WACCM', 'CESM2', 'NorCPM1',
       'GFDL-CM4', 'SAM0-UNICON']
      
array(['MIROC6', 'BCC-ESM1', 'CanESM5-CanOE', 'CanESM5', 'ACCESS-CM2',
       'ACCESS-ESM1-5', 'EC-Earth3-Veg', 'EC-Earth3', 'MPI-ESM-1-2-HAM',
       'IPSL-CM6A-LR', 'MIROC-ES2L', 'HadGEM3-GC31-LL', 'UKESM1-0-LL',
       'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'GISS-E2-1-G-CC',
       'GISS-E2-1-G', 'GISS-E2-1-H', 'CESM2-FV2', 'CESM2-WACCM-FV2',
       'CESM2-WACCM', 'CESM2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM',
       'GFDL-CM4'], dtype=object)

SyntaxError: invalid syntax (<ipython-input-208-f5da4bd048ae>, line 7)

before request: number of ensemble members available
['AWI-CM-1-1-MR' 'BCC-CSM2-MR' 'BCC-ESM1' 'CAMS-CSM1-0' 'FGOALS-f3-L'
 'CanESM5-CanOE' 'CanESM5' 'CNRM-CM6-1-HR' 'CNRM-CM6-1' 'CNRM-ESM2-1'
 'ACCESS-CM2' 'ACCESS-ESM1-5' 'E3SM-1-0' 'E3SM-1-1-ECA' 'E3SM-1-1'
 'EC-Earth3-Veg' 'EC-Earth3' 'FIO-ESM-2-0' 'MPI-ESM-1-2-HAM' 'INM-CM4-8'
 'IPSL-CM6A-LR' 'MIROC-ES2L' 'MIROC6' 'HadGEM3-GC31-LL' 'HadGEM3-GC31-MM'
 'UKESM1-0-LL' 'MPI-ESM1-2-HR' 'MPI-ESM1-2-LR' 'MRI-ESM2-0'
 'GISS-E2-1-G-CC' 'GISS-E2-1-G' 'GISS-E2-1-H' 'CESM2-FV2'
 'CESM2-WACCM-FV2' 'CESM2-WACCM' 'CESM2' 'NorCPM1' 'NorESM2-LM'
 'NorESM2-MM' 'GFDL-CM4' 'GFDL-ESM4' 'NESM3' 'SAM0-UNICON' 'MCM-UA-1-0'
 'INM-CM5-0'] ['Omon']
946


experiment_id,historical,ssp585
variable_id,,
thetao,228,80
uo,227,79
vo,216,78


before request: number of ensemble members available
['AWI-CM-1-1-MR' 'BCC-CSM2-MR' 'BCC-ESM1' 'CAMS-CSM1-0' 'FGOALS-f3-L'
 'CanESM5-CanOE' 'CanESM5' 'CNRM-CM6-1' 'CNRM-ESM2-1' 'ACCESS-CM2'
 'ACCESS-ESM1-5' 'E3SM-1-0' 'E3SM-1-1-ECA' 'E3SM-1-1' 'EC-Earth3-Veg'
 'EC-Earth3' 'FIO-ESM-2-0' 'MPI-ESM-1-2-HAM' 'INM-CM4-8' 'IPSL-CM6A-LR'
 'MIROC-ES2L' 'MIROC6' 'HadGEM3-GC31-LL' 'HadGEM3-GC31-MM' 'UKESM1-0-LL'
 'MPI-ESM1-2-HR' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'GISS-E2-1-G-CC'
 'GISS-E2-1-G' 'GISS-E2-1-H' 'CESM2-FV2' 'CESM2-WACCM-FV2' 'CESM2-WACCM'
 'CESM2' 'NorCPM1' 'NorESM2-MM' 'GFDL-CM4' 'GFDL-ESM4' 'NESM3'
 'SAM0-UNICON' 'MCM-UA-1-0' 'CNRM-CM6-1-HR' 'INM-CM5-0'] ['Omon']
944


experiment_id,historical,ssp585
variable_id,,
thetao,229,79
uo,230,79
vo,218,78


before request: number of ensemble members available
['CanESM5-CanOE' 'CanESM5' 'CNRM-ESM2-1' 'ACCESS-ESM1-5' 'MPI-ESM1-2-HR'
 'IPSL-CM6A-LR' 'MIROC-ES2L' 'UKESM1-0-LL' 'MPI-ESM1-2-LR' 'CESM2-WACCM'
 'CESM2' 'NorESM2-LM' 'NorESM2-MM' 'GFDL-ESM4' 'MRI-ESM2-0' 'GFDL-CM4'
 'CNRM-CM6-1-HR' 'CNRM-CM6-1' 'ACCESS-CM2' 'EC-Earth3-Veg' 'EC-Earth3'
 'BCC-CSM2-MR' 'INM-CM4-8' 'INM-CM5-0'] ['Omon']
823


experiment_id,ssp585
variable_id,
chldiatos,9
chlmisc,4
chlmiscos,6
chlpico,7
dfe,16
dfeos,14
dissicnatos,5
epc100,83
fgco2,97


before request: number of ensemble members available


experiment_id,ssp585
variable_id,
chlmisc,3
chlpico,7
dfe,10
dfeos,1
epc100,78
fgco2,93
intpoc,1
intpp,38
intppdiaz,1


before request: number of ensemble members available


variable_id,chlmisc,chlpico,dfe,dfeos,epc100,fgco2,intpoc,intpp,intppdiaz,nh4,nh4os,no3,no3os,phyc,phydiat,phydiaz,phydiazos,phymisc,phypico,po4,po4os,pp,rsntds,si,spco2,zooc
experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,
ssp585,3,7,10,1,78,93,1,38,1,5,1,52,2,11,4,3,1,6,1,32,1,21,14,6,84,9


before request: number of ensemble members available


variable_id,tas,tasmax,tasmin
experiment_id,,,
amip,104,68,70
historical,313,248,252


before request: number of ensemble members available


variable_id,chl,dissic,dissoc,icfriver,o2sat,ocfriver
experiment_id,,,,,,
1pctCO2,31,31,17,10,10,10
esm-hist,27,28,13,11,12,11
historical,147,154,85,53,52,53
ssp245,73,73,41,25,30,25
ssp370,82,82,51,35,39,35
ssp585,67,67,34,25,28,24


before request: number of ensemble members available


variable_id,chl,dissic,dissoc,icfriver,o2sat,ocfriver
experiment_id,,,,,,
1pctCO2,21,20,13,2,2,2
esm-hist,14,15,3,0,2,1
historical,80,106,24,5,17,5
ssp245,61,61,30,2,20,2
ssp370,8,7,5,3,6,3
ssp585,55,54,23,3,17,2


before request: number of ensemble members available


variable_id,pr,prsn,tas
experiment_id,,,
historical,169,28,178


In [93]:
dm = df[['experiment_id','source_id','variable_id','member_id']].groupby([
         'experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm,
                                 values='member_id',
                                 index=['experiment_id','source_id'],
                                 columns=['variable_id'],
                                 aggfunc=np.sum,
                                 fill_value=0)

print('current CMIP6 datasets')
display(table)

current CMIP6 datasets


variable_id                  chlmisc  chlpico  dfe  dfeos  epc100  fgco2  intpoc  intpp  intppdiaz  nh4  nh4os  no3  no3os  phyc  phydiat  phydiaz  phydiazos  phymisc  phypico  po4  po4os  pp  rsntds  si  spco2  zooc
experiment_id source_id                                                                                                                                                                                                 
ssp585        ACCESS-CM2           0        0    0      0       0      0       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       1   0      0     0
              ACCESS-ESM1-5        0        0    1      0       3      3       0      3          0    0      0    2      0     0        0        0          0        0        0    3      0   0       1   0      3     0
              BCC-CSM2-MR          0        0    0      0       0      1       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       0   0      0     0
              CESM2                0        2    0      0       2      2       0      2          0    0      0    2      0     0        0        0          0        0        0    2      0   2       0   0      2     0
              CESM2-WACCM          0        5    0      0       4      5       0      4          0    1      1    4      1     0        0        0          0        0        0    5      1   4       0   0      4     0
              CNRM-CM6-1-HR        0        0    0      0       0      0       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       1   0      0     0
              CNRM-ESM2-1          1        0    1      0       5      5       0      5          0    1      0    5      0     1        1        0          0        1        0    5      0   5       1   1      5     1
              CanESM5              0        0    0      0      50     50       0      5          0    0      0   20      0     2        0        0          0        0        0    0      0   0       2   0     50     2
              CanESM5-CanOE        1        0    1      0       3      3       0      3          0    1      0    3      0     1        1        0          0        1        0    0      0   0       1   0      3     1
              EC-Earth3            0        0    0      0       0      0       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       1   0      0     0
              EC-Earth3-Veg        0        0    0      0       0      0       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       1   0      0     0
              GFDL-CM4             0        0    1      0       0      1       0      0          0    0      0    0      0     1        0        0          0        0        0    1      0   1       0   0      1     0
              GFDL-ESM4            0        0    1      0       1      1       0      1          0    1      0    1      0     1        1        1          1        1        1    1      0   1       0   1      1     1
              INM-CM4-8            0        0    0      0       0      1       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       0   0      0     0
              INM-CM5-0            0        0    0      0       0      1       0      0          0    0      0    0      0     0        0        0          0        0        0    0      0   0       0   0      0     0
              IPSL-CM6A-LR         1        0    1      0       6      6       0      6          0    1      0    6      0     1        1        0          0        1        0    6      0   6       1   1      6     1
              MIROC-ES2L           0        0    1      0       1      1       0      1       

In [ ]:
df.variable_id.unique()

In [ ]:
table[(table.chl>0)&(table.dissic>0)]

In [ ]:
#xr.open_zarr('/h63/naomi/zarr-minimal/AerChemMIP/MOHC/UKESM1-0-LL/piClim-control/r1i1p1f2/AERmon/mmroa/gn')

In [ ]:
xr.open_zarr('gs://cmip6/ScenarioMIP/CNRM-CERFACS/CNRM-ESM2-1/ssp585/r1i1p1f2/E3hr/uas/gr/')